<a href="https://colab.research.google.com/github/HAL22/Kaggle-Competitions/blob/Disaster-Tweets/Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Importing python data science and huggingface libraries

In [ ]:
%%capture
!pip install kaggle
import pandas as pd
!pip install datasets transformers[sentencepiece]
!apt install git-lfs

In [ ]:
from datasets import load_dataset
from datasets import Dataset, DatasetDict, load_dataset
from sklearn.model_selection import train_test_split

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!git config --global user.email "thethelafaltein@gmail.com"
!git config --global user.name "HAL22"

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


2. Getting the Kaggle dataset

In [ ]:
%%capture
!kaggle competitions download -c nlp-getting-started
!unzip /content/nlp-getting-started.zip

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


3. Preparing/Formatting the data for the model

In [ ]:
labels = []
for index, row in df_train.iterrows():
  value = df_train.iloc[index]['target']
  if value == 1:
    labels.append(0)
  else:
    labels.append(1)  

In [ ]:
df_train['label']= labels

In [ ]:
df_train.head()

,id,keyword,location,text,target,label
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0


In [ ]:
df_train = df_train[['text', 'label']].copy()

In [ ]:
df_train

,text,label
0,Our Deeds are the Reason of this #earthquake M...,0
1,Forest fire near La Ronge Sask. Canada,0
2,All residents asked to 'shelter in place' are ...,0
3,"13,000 people receive #wildfires evacuation or...",0
4,Just got sent this photo from Ruby #Alaska as ...,0
...,...,...
7608,Two giant cranes holding a bridge collapse int...,0
7609,@aria_ahrary @TheTawniest The out of control w...,0
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,0
7611,Police investigating after an e-bike collided ...,0


In [ ]:
df_train, df_test = train_test_split(df_train, test_size=0.2)

In [ ]:
df_validation = pd.read_csv('test.csv')
df_validation = df_validation[['text']].copy()
validation = Dataset.from_pandas(df_validation)

In [ ]:
train = Dataset.from_pandas(df_train)
test = Dataset.from_pandas(df_test)

In [ ]:
train

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 6090
})

In [ ]:
dataset = DatasetDict()
dataset['train'] = train
dataset['test'] = test

In [ ]:
dataset['validation'] = validation

4. Installing the model

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment", num_labels=3)

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6090
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1905
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a met

Step,Training Loss
500,0.409800
1000,0.287300
1500,0.199800


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1500/special_toke

TrainOutput(global_step=1905, training_loss=0.264779823283198, metrics={'train_runtime': 374.4325, 'train_samples_per_second': 81.323, 'train_steps_per_second': 5.088, 'total_flos': 853072284121092.0, 'train_loss': 0.264779823283198, 'epoch': 5.0})

4. Evaluating the model

In [ ]:
df_validation = pd.read_csv('test.csv')

In [ ]:
df_validation = df_validation[['text']].copy()

In [ ]:
df_validation.head()

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
validation = Dataset.from_pandas(df_validation)

In [ ]:
validation

Dataset({
    features: ['text'],
    num_rows: 3263
})

In [ ]:
dataset['validation'] = validation

In [ ]:
predictions = trainer.predict(tokenized_dataset["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3263
  Batch size = 16


AttributeError: ignored

In [ ]:
predictions

PredictionOutput(predictions=None, label_ids=None, metrics={'test_runtime': 0.0217, 'test_samples_per_second': 0.0, 'test_steps_per_second': 0.0})